List of thing to do:

- Explain what does it mean the arguments in the main function

- Basic C functions (Memcpy, Malloc)
- Kernels
- Launching a kernel
- Maybe: ways to split our information

# Elevando al cuadrado en serie y en paralelo

Nuestra primer misión en CUDA es explicar ciertas sutilezas, para esto iniciaremos con un pequeño ejemplo.

Supongamos que queremos elevar al cuadrado los números del 1 al 100. Un programa en python podríamos hacerlo de la siguiente manera.

- Generamos un arreglo con los números del 1 al 100.
- Utilizando un ciclo **for** elevamos cada uno de estos números y los metemos en un nuevo arreglo.

Por último lo imprimos para mostrarlo al lector.

In [1]:
enteros = range(1,101)
cuadrados = []

for i in enteros:
    cuadrados.append(i*i)

print(cuadrados)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801, 10000]


Como podemos ver, también hemos calculado el tiempo que se toma en hacer todo el proceso. Analicemos un poco nuestro algoritmo; el programa hace lo siguiente

- Lee un número del arreglo.
- Lo multiplica por sí mismo.
- Guarda el resultado en un nuevo arreglo.
- Si hay un siguiente número se regresa al punto inicial, si no termina.

Supongamos a manera de esquema que cada multiplicación en nuestro procesador toma **2 ns** en realizarse, como estamos haciendo 100 multiplicaciones, entonces nos tomaría **200 ns** obtener todos y cada uno de los cuadrados que queremos.

¿Cómo podríamos hacer esto en paralelo?

Bueno, tal vez utilizando 100 trabajadores (a los cuales de aquí en adelante llamaremos threads o hilos) y haciendo que cada uno se encargue de leer un número del arreglo de entrada, multiplicarlo por sí mismo y escribirlo dentro de otro arreglo. Si vemos lo que hace un sólo thread no podríamos notar cuál es la diferencia entre el algoritmo en Python (algoritmo escrito en serie) y el algoritmo en parelelo. La principal diferencia radica en que en el algoritmo serial un solo thread se encarga de multiplicar todos los números, avanzando de uno en uno, mientras que en el algoritmo en paralelo le damos un número a cada uno de los 100 trabajadores y los hacemos multiplicarlos por sí mismos. Si cada uno de estos trabajadores tardara **10 ns** en hacer una multiplicación ¿Cuánto se tardarían en multiplicar los 100 números? Exacto, **10 ns**.

Ahora veamos cómo quedaría éste código escrito en CUDA, recordando que **al final de cada declaracion se debe poner un punto y coma.**

```C
#include <stdio.h>

__global__ void elevar_al_cuadrado(float * d_salida, float * d_entrada){
	int idx = threadIdx.x;
    float f = d_entrada[idx];
    d_salida[idx] = f*f;
}

int main(int argc, char ** argv) {

	const int TAMANIO_ARREGLO = 100;
	const int BYTES_ARREGLO = TAMANIO_ARREGLO * sizeof(float);

	// Generamos el arreglo de entrada en el anfitrion
	float h_entrada[TAMANIO_ARREGLO];
    
	for (int i = 0; i < TAMANIO_ARREGLO; i++) {
		h_entrada[i] = float(i);
	}
    
	float h_salida[TAMANIO_ARREGLO];

	// Declaramos apuntadores de memoria en GPU
	float * d_entrada;
	float * d_salida;

	// Reservamos memoria del GPU
	cudaMalloc((void**) &d_entrada, BYTES_ARREGLO);
	cudaMalloc((void**) &d_salida, BYTES_ARREGLO);

	// Copiamos informacion al GPU
	cudaMemcpy(d_entrada, h_entrada, BYTES_ARREGLO, cudaMemcpyHostToDevice);

	// Lanza el kernel
	elevar_al_cuadrado<<<1, TAMANIO_ARREGLO>>>(d_salida, d_entrada);

	// Copiamos el arreglo resultante al GPU
	cudaMemcpy(h_salida, d_salida, BYTES_ARREGLO, cudaMemcpyDeviceToHost);

	// Imprimimos el arreglo resultante
	for (int i =0; i < TAMANIO_ARREGLO; i++) {
		printf("%f", h_salida[i]);
		printf(((i % 4) != 3) ? "\t" : "\n");
	}

	cudaFree(d_entrada);
	cudaFree(d_salida);

	return 0;
}```

No tan simple como en Python ¿cierto?

La parte 
```C
#include <stdio.h>```
simplemente agrega a nuestro programa la librería básica de C, que contiene funciones como ```printf()```, ```scanf()```, etc. Esto no es en el sentido extricto algo relacionado con CUDA, sin embargo, al estar basado en C CUDA sigue reglas similares.

Lo siguiente en nuestro código es

```C
__global__ void elevar_al_cuadrado(float * d_salida, float * d_entrada){
	int idx = threadIdx.x;
    float f = d_entrada[idx];
    d_salida[idx] = f*f;
}```

Esto es lo que conocemos como kernel; en el kernel se ponen los comandos que el programador desea que ejecute cada uno de los threads (nuestros [Oompa Loompas](https://es.wikipedia.org/wiki/Oompa_Loompa) del GPU). A primera vista, y como ya habíamos adelantado, las órdenes parecen ser de un código serial, en ninguna parte podemos ver exactamente donde yace el paralelismo, pero hay que tener calma, llegaremos a ese punto en este mismo notebook.

Vayamos paso a paso, esta parte del código simplemente define una función para elevar al cuadrado números en un arreglo. El lector familiarizado con C habrá notado una peculiaridad (y el lector no familiarizado está a punto de enterarse), en C no existe tal cosa como ```__global__``` (con dos guiones bajos a cada lado); nos encontramos pues ante la primer característica de CUDA.

##Calificadores de tipo de función

Al ser un lenguaje de programación diseñado para controlar tanto al CPU como al GPU CUDA debe saber de alguna forma si la función que estamos escribiendo se va a ejecutar en uno u otro. Es por eso que cuenta con ciertos [calificadores de tipo de función](http://docs.nvidia.com/cuda/cuda-c-programming-guide/#function-type-qualifiers). Los tres principales son:

* ```__global__``` : especifica que la función es un kernel. Dicha función es
 - Ejecutada en el dispositivo (GPU)
 - Invocable desde el anfitrión (CPU)
 - Invocable desde el dispositivo para dispositivos (para quien tenga más de un  GPU)
 
 Cuenta también con ciertas cualidades; debe ser una función de tipo ```void```, es decir, que no regresa nada. Y además las invocaciones a ```__global__``` son asíncronas, en otras palabras, la función devuelve el control al programa principal antes de haber terminado de hacer el trabajo.
 
 
* ```__device__``` : este calificador declara una función que es

 - Ejecutada en el dispositivo.
 - Invocable solamente desde el dispositivo.
 
 
* ```__host__``` : declara una función

 - Ejecutada en el anfitrión.
 - Invocable solamente desde el anfitrión.
 
Lo siguiente en la lista es ```void```, esto lo que quiere decir es que la función no va a regresar ningún tipo de dato, o lo que es lo mismo, sólo va a trabajar. En ocasiones es fácil confundirse con este concepto, si no va a regresar nada ¿por qué hacerlo?. Pongamos dos situaciones distintas, en la primera quieres saber cuál es el resultado de hacer cierta operación e imprimirlo, por ejemplo saber cuánto es 4!. Pero bien puede suceder que después necesites ese valor. En el caso en que sólo deseas saber cuánto vale, el tipo de función que se necesita es ```void```, ya que sólo queremos que calcule 4! y nos diga cuánto es. En el segundo caso usaríamos una función de tipo ```int``` (integer) que regrese un número entero. La diferencia sería que la segunda función puede ser utilizada como cualquier número entero, la podemos multiplicar, sumar, dividir... A la primera no.

Después tenemos el nombre de la función, que en nuestro caso es ```elevar_al_cuadrado``` y recibe dos argumentos, declarados como 
```C 
float * d_entrada```
```C
float * d_salida```

Por convención, las variables que serán usadas por el dispositivo (device) se marcan con el prefijo ```d_```
mientras que las utilizadas por el anfitrión (host) llevan el prefijo ```h_```. Así pues, vemos que los dos argumentos que recibe nuestra función son *bichos* alojados en el dispositivo, dos bichos que reconocemos como números de punto flotante, pero; hay un asterisco que nos estorba, **¿será esto causante de una nueva sección en las notas?**

##Apuntadores

Un apuntador es una variable cuyo valor es la dirección en memoria de otra variable. Supongamos que tenemos lo siguiente en C

```C
 float pi = 3.1416;
 float * apuntador_pi;
 
 apuntador_pi = &pi;
```

En el primer renglón declaramos un flotante llamado pi con cierto valor, en el siguiente declaramos un apuntador que será dirigido hacia un número flotante, y por último dirigimos el apuntador hacia el valor deseado, usando el caracter &.

Si quisiésemos imprimir el valor de ```apuntador_pi``` la computadora escupiría algo como ```bffd8b3c``` mientras que al imprimir ```*apuntador_pi``` obtenemos ```3.1416```.

La pregunta que nace naturalmente es, ¿por qué complicarse la vida con apuntadores? Bueno, los apuntadores son necesarios cuando se necesita reservar memoria dinámicamente, o cuando se manejan cantidades grandes de información. Los apuntadores fueron creados en una época en que se necesitaba un uso de memoria rápido y eficiente dado que las máquinas eran demasiado lentas, eso quiere decir que en la actualidad podemos usar apuntadores para hacer correr más rápido nuestros programas o *usar menos memoria*, esto último, como veremos más adelante es una característica deseable cuando utilizamos GPU.

##De regreso al código

Entonces, hasta ahora hemos visto que en el primer renglón se declaró una función, ejecutable en el dispositivo pero invocable solamente desde el anfitrión, que no regresa ningún tipo de dato, llamada ```elevar_al_cuadrado```, que recibe como argumentos dos apuntadores a estructuras alojadas en el dispositivo (demasiado para un renglón si me preguntan).

Lo siguiente en la lista es 

```C
	int idx = threadIdx.x;
```

Declaramos una variable llamada ```ìdx``` y le asignamos el valor de ```threadIdx.x```, como recordará el lector, a nuestros trajadores le habíamos cambiado el nombre por *threads*, ```threadIdx.x``` simplemente nos dice cuál es la etiqueta que porta el thread, o el número de trabajador si se quiere ver de otra forma; dicho número es exclusivo para cada uno de los threads, y como podemos ver, con el uso del sufijo ```.x``` los threads tienen etiquetas en cada dimensión. Es decir que podríamos acomodar threads en una cuadrícula tridimensional en donde cada uno estaría al tanto de su posición en x, y, z.

Finalmente tenemos

```C
    float f = d_entrada[idx];
    d_salida[idx] = f*f;
```
En la primer parte declaramos una variable ```f``` que guardará el valor de la entrada ubicada en la posición ```idx``` del arreglo cuyo apuntador se metió como argumento a la función. En la segunda elevamos el número al cuadrado y lo guardamos en la entrada ```idx``` del segundo arreglo cuyo apuntador se ingresa en la funcion como argumento.

Como prometimos esta función (o kernel), será ejecutado en el GPU. Le pedimos (de una forma conveniente) 100 trabajadores al dispositivo y hacemos que cada uno de ellos ejecute el kernel. Ya antes mencionamos que cada uno de ellos contará con un índice (número de trabajador) único, y por lo tanto leerán una entrada específica del arreglo de entrada y escribirán en una entrada específica del arreglo de salida, por lo tanto no habrá que temer que dos threads intenten leer y escribir en los mismos sitios. Más adelante veremos que éste tipo de proceso es conocido como **mapeo** en el contexto de patrones de comunicación.

##Avanzamos en el código

Hasta ahora hemos visto cómo se especifica cuál es el trabajo que hará cada uno de los threads, sin embargo, en ningún momento los pusimos a trabajar ¿cómo es que se logra esto? ¿nuestro kernel sirve para el número de threads que deseemos? Es hora de explicar con detenimiento el proceso de ejecución del kernel.

```
no tengo la más minima idea de por qué está eso en el main, no sé si es estándar
```

La siguiente parte del código es simple de entender, más no intuitiva.

```C
	const int TAMANIO_ARREGLO = 100;
	const int BYTES_ARREGLO = TAMANIO_ARREGLO * sizeof(float);
```

El primer renglón simplemente define el tamaño del arreglo que usaremos. En nuestro caso, dado que queremos obtener los cuadrados de los 100 primeros números naturales (consideramos que el 0 es un número natural) declaramos una constante que guarde el tamaño del arreglo. En el segundo renglón obtenemos el tamaño en bytes del arreglo y lo guardamos en una nueva variable. Es importante observar aquí que no supusimos ningún tamaño específico para los números flotantes, sino que lo obtuvimos utilizando la función ```sizeof()```, al momento de programar es importante escribir de tal manera que nuestros programas puedan ser corridos en otras computadoras. 

Quizá se estarán preguntando ¿para qué guardar el tamaño del arreglo en bytes? Puede parecer un poco inútil, sin embargo detrás de éste renglón se esconde una sutileza importante de CUDA. **No, no es el hecho de que escribimos "ni" en lugar de "ñ"**

Después declaramos un arreglo de números flotantes, lo llenamos con los números del 0 al 99, y declaramos otro arreglo de flotantes para almacenar en él los cuadrados.

```C
	float h_entrada[TAMANIO_ARREGLO];
	for (int i = 0; i < TAMANIO_ARREGLO; i++) {
		h_entrada[i] = float(i);
	}
	float h_salida[TAMANIO_ARREGLO];
```

*Podría el lector decir ¿quién se utilizará esos arreglos?*

#Hacer una celda con widgets con dos botones.